# Web Scraping

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
# Define executable path and browser for splinter
executable_path = {'executable_path':'/chromedriver/chromedriver.exe'}
browser = Browser('chrome',**executable_path, headless=False)

## NASA Mars News

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [9]:
# Get a handle on the URL with splinter using a wait_time to allow for all results to come through
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=0.5)
browser.visit(url)

In [10]:
# Assign the splinter handle to an 'html' object and parse it with BeautifulSoup
html = browser.html
soup = bs(html,'lxml')

In [11]:
# Retrieve the most recent headlines in the page
headlines = soup.find('div', class_='list_text')
print(headlines)

<div class="list_text"><div class="list_date">July 23, 2020</div><div class="content_title"><a href="/news/8719/nasa-invites-public-to-share-excitement-of-mars-2020-perseverance-rover-launch/" target="_self">NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch</a></div><div class="article_teaser_body">There are lots of ways to participate in the historic event, which is targeted for July 30.</div></div>


In [12]:
# Retrieve the most recent headline title and paragraph text
news_title = headlines.find('div', class_='content_title').a.text
news_p = headlines.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch
There are lots of ways to participate in the historic event, which is targeted for July 30.


## JPL Mars Space Images - Featured Images

In [22]:
# URL of page to be scraped
url_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [35]:
# Get a handle on the URL with splinter using a wait_time to allow for all results to come through
browser.is_element_present_by_css("section.primary_media_feature", wait_time=0.5)
browser.visit(url_image)

In [36]:
# Assign the splinter handle to an 'html' object and parse it with BeautifulSoup
html_image = browser.html
soup_image = bs(html_image,'lxml')

In [48]:
# Select the most recent image and click on it to go to the large version
featured_image_url = soup_image.find('footer').find('a', class_='button')['data-fancybox-href']
featured_image_url = featured_image_url.strip('/')
featured_image_url = f'https://www.{featured_image_url}'
print(featured_image_url)

https://www.spaceimages/images/mediumsize/PIA18289_ip.jpg
